<a href="https://colab.research.google.com/github/dipnarayan501/Deep_learning/blob/main/CS6910_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import mnist, fashion_mnist
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
%pip install wandb -q
import wandb
wandb.login()

wandb: Currently logged in as: moni6264 (use `wandb login --relogin` to force relogin)


True

In [3]:
def prepare_dataset():
    (X, y), (X_test, y_test) = fashion_mnist.load_data()

    # Reshaping the data matrices
    X = X.reshape(X.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)

    # Normalizing the pixel intensities
    X = X/255.0
    X_test = X_test/255.0
    
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
    
    y_train_encode = np.zeros((10,y_train.shape[0]))
    y_train_encode[y_train, np.array(list(range(y_train.shape[0])))] = 1

    y_val_encode = np.zeros((10,y_val.shape[0]))
    y_val_encode[y_val, np.array(list(range(y_val.shape[0])))] = 1

    
    
    print("Number of examples in training set")
    print(X_train.shape)
    print(y_train_encode.shape)
    print(y_train.shape)
    
    print("No of examples in validation set")
    print(X_val.shape)
    print(y_val.shape)
    print("No of examples in test set")
    print(X_test.shape)
    print(y_test.shape)
    
    return X_train.T, y_train_encode, y_train.T ,X_val.T, y_val.T, X_test.T, y_test.T, y_val_encode


X_train, y_train_encode, y_train ,X_val, y_val, X_test, y_test,y_val_encode = prepare_dataset() 

Number of examples in training set
(54000, 784)
(10, 54000)
(54000,)
No of examples in validation set
(6000, 784)
(6000,)
No of examples in test set
(10000, 784)
(10000,)


In [5]:
num_classes = 10
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
               
wandb.init(project="test_100", entity="moni6264", name="log_images")
indices = [list(y_train.T).index(i) for i in range(num_classes)]
images = []
captions = []
for index in indices:
    images.append(X_train.T[index].reshape((28, 28)))
    captions.append(class_names[y_train.T[index]])

# Log one sample image of each class to wandb
wandb.log({"Image from each class": [wandb.Image(image, caption=caption) for image, caption in zip(images, captions)]})

In [6]:
def sigmoid(x):
    return 1. / (1.+np.exp(-x))
def relu(x):
    return np.maximum(0,x)
def tanh(x):
    return np.tanh(x)
def softmax(x):
    soft = np.zeros(x.shape)
    for i in range(0, x.shape[1]):
        numr = np.exp(x[:, i])
        soft[:, i] = numr/np.sum(numr)
    return soft

#Derivatives
def sigmoid_derivative(x):
    return sigmoid(x) * (1-sigmoid(x))
def relu_derivative(x):
    return 1*(x>0) 
def tanh_derivative(x):
    return (1 - (np.tanh(x)**2))

def softmax(x):
    soft = np.zeros(x.shape)
    for i in range(0, x.shape[1]):
        numr = np.exp(x[:, i])
        soft[:, i] = numr/np.sum(numr)
    return soft

def initialize_parameters(input_layer, hidden_layer, output_layer,initilisation):
    weights = []
    biases  = []
    layers = [input_layer] + hidden_layer + [output_layer]
    for i in range(len(hidden_layer)+1):
        if initilisation == 'random':
            weights.append(np.random.rand(layers[i+1], layers[i])*0.01)
        if initilisation == 'xavier':
            weights.append(np.random.randn(layers[i+1],layers[i])*np.sqrt(2/layers[i+1]))
        biases.append(np.random.randn(layers[i+1],1))
    return weights, biases

In [7]:
import numpy as np
from sklearn.metrics import mean_squared_error

class ANN:

    def __init__(self,input_layer,hidden_layer,output_layer ,initialisation, activation,loss_function):
           
        self.input_layer   = input_layer
        self.output_layer  = output_layer
        self.hidden_layer   = hidden_layer
        self.activation = activation
        self.loss_function = loss_function
        self.weights    = []
        self.biases     = []
        self.initialisation = initialisation
        self.weights,self.biases = initialize_parameters(self.input_layer,self.hidden_layer,self.output_layer,self.initialisation)

    
    

    def forward_propagation(self,input):

        self.A = []
        self.H  = []
        k=0
        
        self.A.append(np.matmul(self.weights[k],input)+self.biases[k])
        if self.activation == 'sigmoid':
          self.H.append(sigmoid(self.A[k]))
        elif self.activation == 'tanh':
          self.H.append(tanh(self.A[k]))
        elif self.activation == 'relu': 
          self.H.append(relu(self.A[k])) 


        for k in range(1,len(self.hidden_layer)):
            self.A.append(np.matmul(self.weights[k],self.H[k-1])+self.biases[k])
            if self.activation == 'sigmoid':
              self.H.append(sigmoid(self.A[k]))
            elif self.activation == 'tanh':
              self.H.append(tanh(self.A[k]))
            elif self.activation == 'relu': 
              self.H.append(relu(self.A[k])) 
  


        k=len(self.hidden_layer)
        self.A.append(np.matmul(self.weights[k],self.H[k-1])+self.biases[k])
        self.H.append(softmax(self.A[k]))

        return self.H[-1]

    def back_propagation(self,X_train,y_train):
        g_a  = [0]*(len(self.hidden_layer)+1)
        g_h  = [0]*(len(self.hidden_layer)+1)
        g_w  = [0]*(len(self.weights))
        g_b  = [0]*(len(self.biases))

        n_samples = X_train.shape[1]  # Change depending on the dimensions of data


        for k in reversed(range(len(self.hidden_layer)+1)):
            if k == len(self.hidden_layer):
              if self.loss_function == 'cross_entropy':
                  g_a[k] = self.H[k]  - y_train  # keep or remove T depending on the dimensions of data
              elif self.loss_function == 'square_loss': 
                  g_a[k] = (self.H[k] - y_train) * self.H[k] * (1 - self.H[k]) 
                
            else:
                g_h[k] = (1/n_samples)*np.matmul(self.weights[k+1].T,g_a[k+1])
                if self.activation == 'sigmoid':
                  g_a[k] = (1/n_samples)*np.multiply(g_h[k],sigmoid_derivative(self.A[k]))
                elif self.activation == 'tanh':
                  g_a[k] = (1/n_samples)*np.multiply(g_h[k],tanh_derivative(self.A[k]))
                elif self.activation == 'relu':
                  g_a[k] = (1/n_samples)*np.multiply(g_h[k],relu_derivative(self.A[k]))
                

            if k == 0:
                g_w[k] = (1/n_samples)*np.matmul(g_a[k],X_train.T) 
            else:
                g_w[k] = (1/n_samples)*np.matmul(g_a[k],self.H[k-1].T)

            g_b[k]  = (1/n_samples)*np.sum(g_a[k], axis=1, keepdims = True)
        return g_w,g_b


    def predict(self, X,y ):
      output =  self.forward_propagation(X)
      out_class=(np.argmax(output,axis=0))
      accuracy = round(accuracy_score(y, out_class) * 100, 3)
      return accuracy
    def fit_predict(self,X_train,y_train,X_val ,y_val ,epochs,learning_rate,optimiser='gd',batch_size = 64,lambd=0.0005): 
      steps = 0
      pre_update_w = np.multiply(self.weights,0)
      pre_update_b = np.multiply(self.biases,0)
      update_w = np.multiply(self.weights,0)
      update_b = np.multiply(self.biases,0)
      vw = 0.0
      vb = 0.0
      eps = 1e-8
      a1 =0.0
      gamma = 0.9
      beta = 0.999
      beta1 = 0.9
      beta2 = 0.999
      m_t, v_t, m_hat_w, v_hat_w, m_b,v_b,m_hat_b,v_hat_b = 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0 

      for i in range(epochs):
        

            for batch in range(0, X_train.shape[1], batch_size):

              train_b_imag =  X_train[:,batch:batch+batch_size]
              train_l_imag =  y_train_encode[:,batch:batch+batch_size]
              output =  self.forward_propagation(train_b_imag)
              g_weights,g_biases = self.back_propagation(train_b_imag,train_l_imag)
              if optimiser == 'gd':
                 self.weights = self.weights - np.multiply(learning_rate,g_weights) - np.multiply(learning_rate*lambd,self.weights)
                 self.biases = self.biases - np.multiply(learning_rate,g_biases)
             
              if optimiser == 'mgd':
                 
                 update_w = np.multiply(gamma,pre_update_w) + np.multiply(learning_rate,g_weights)
                 self.weights = self.weights - update_w - np.multiply(learning_rate*lambd,self.weights)
                
                 update_b = np.multiply(gamma,pre_update_b) + np.multiply(learning_rate,g_biases)
                 self.biases = self.biases - update_b
                 pre_update_w = update_w
                 pre_update_b = update_b

              if optimiser == 'ngd':
                 
                 cache_w = self.weights
                 cache_b = self.biases
                 self.weights = self.weights - np.multiply(gamma,pre_update_w)
                 self.biases = self.biases - np.multiply(gamma,pre_update_b)
                 output =  self.forward_propagation(train_b_imag)
                 g_weights,g_biases = self.back_propagation(train_b_imag,train_l_imag)

                 update_w = np.multiply(gamma,pre_update_w) + np.multiply(learning_rate,g_weights)
                 cache_w = cache_w - update_w - np.multiply(learning_rate*lambd,cache_w)
                    
                 update_b = np.multiply(gamma,pre_update_b) + np.multiply(learning_rate,g_biases)
                 cache_b = cache_b - update_b
                 
                 self.weights = cache_w
                 self.biases = cache_b
                 pre_update_w = update_w
                 pre_update_b = update_b
                
              if optimiser == 'rmsprop': 

                 vw = np.multiply(vw,beta) + np.multiply(1-beta,np.power(g_weights,2))
                 vb = np.multiply(vb,beta) + np.multiply(1-beta,np.power(g_biases,2))

          
                 update_w = np.multiply(g_weights,learning_rate/np.power(vw+eps,1/2))
                 update_b = np.multiply(g_biases,learning_rate/np.power(vb+eps,1/2))
                 
                 self.weights = self.weights - update_w - np.multiply(learning_rate*lambd,self.weights)
                 self.biases = self.biases - update_b
                    
              if optimiser == 'adam':
                 m_t = np.multiply(beta1,m_t) + np.multiply(1-beta1,g_weights)
                 v_t = np.multiply(beta2,v_t) + np.multiply(1-beta2,np.power(g_weights,2))
                 m_b = np.multiply(beta1,m_b) + np.multiply(1-beta1,g_biases)
                 v_b = np.multiply(beta2,v_b) + np.multiply(1-beta2,np.power(g_biases,2))
                
                 m_hat_w = m_t/(1 - np.power(beta1,i+1))
                 m_hat_b = m_b/(1 - np.power(beta1,i+1))
                
                 v_hat_w = v_t/(1 - np.power(beta2,i+1))
                 v_hat_b = v_b/(1 - np.power(beta2,i+1))
                 update_w = (learning_rate / np.power(v_hat_w + eps, 1/2)) * m_hat_w
                 update_b = (learning_rate / np.power(v_hat_b + eps, 1/2)) * m_hat_b
                 self.weights = self.weights - update_w - np.multiply(learning_rate*lambd,self.weights)
                 self.biases = self.biases - update_b

              if optimiser == 'nadam':
                 
                 self.weights = self.weights - np.multiply(gamma,update_w)
                 self.biases  = self.biases  - np.multiply(gamma,update_b)

                 g_weights,g_biases = self.back_propagation(train_b_imag,train_l_imag)

                 m_t =  np.multiply(beta1,m_t) + np.multiply(1 - beta1,g_weights)
                 v_t =  np.multiply(beta2,v_t) + np.multiply(1 - beta2,np.power(g_weights, 2))

                 m_b =  np.multiply(beta1,m_b) + np.multiply(1 - beta1,g_biases)
                 v_b =  np.multiply(beta2,v_b) + np.multiply(1 - beta2,np.power(g_biases, 2))
                
                 m_hat_w = m_t / (1 - np.power(beta1, i+1)) 
                 v_hat_t = v_t / (1 - np.power(beta2, i+1))

                 m_hat_b = m_b / (1 - np.power(beta1, i+1)) 
                 v_hat_b = v_b / (1 - np.power(beta2, i+1))
  
                 a1 = (1-beta1)/(1-np.power(beta,i+1))
                 update_w = np.multiply(learning_rate/(np.power(v_hat_t + eps,1/2)),(np.multiply(a1,g_weights) + np.multiply(beta1,m_hat_w)))#(a4 + a2))
                 update_b = np.multiply(learning_rate/(np.power(v_hat_b + eps,1/2)),(np.multiply(a1,g_biases)+np.multiply(beta1,m_hat_b) ))#(a5 + a3))
                 self.weights = self.weights - update_w - np.multiply(learning_rate*lambd,self.weights)
                 self.biases = self.biases - update_b
                
            
            #Training loss for full dataset
            predicted_train = self.forward_propagation(X_train)
            predicted_train_label=(np.argmax(predicted_train,axis=0)) 
             
            acc1 = round(accuracy_score(y_train, predicted_train_label) * 100, 3)

            predicted_val = self.forward_propagation(X_val)
            predicted_val_label=(np.argmax(predicted_val,axis=0))
              
            acc2 = round(accuracy_score(y_val, predicted_val_label) * 100, 3)

            a =self.weights[1:len(self.hidden_layer)]
            b = np.sum([(np.sum((a[i]**2).reshape(1,-1))) for i in range(len(a))]) 
            if self.loss_function == 'cross_entropy':
              train_loss = (-np.sum(np.multiply(y_train_encode,np.log(predicted_train)))+((lambd/2.)*b))/y_train.shape[0]
              val_loss = (-np.sum(np.multiply(y_val_encode,np.log(predicted_val)))+((lambd/2.)*b))/y_val.shape[0]



            print('Epoch {}: training_accuracy = {:.2f} %, training_loss= {:.5f}  ,Validation accuracy = {:.2f} ,Validation loss = {:.5f}'.format(i,acc1,train_loss,acc2,val_loss))
            



            
            #print("Accuracy:{} epoch  {}".format(i,acc1))
            wandb.log({"val_accuracy": acc2,"train_accuracy": acc1,"steps":epochs,"train_loss":train_loss,"val_loss":val_loss},)
            #run_name = "init_{}_act_{}_lr_{}_opt_{}_L2_{}_bs_{}_ep_{}_hh_{}".format(self.initialisation,self.activation,learning_rate,optimiser,lambd,batch_size, epochs, self.hidden_layer)
            #print(run_name)
            

         
      return acc1,acc2,train_loss,val_loss

In [8]:
%pip install wandb -q
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [9]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize' 
    },
    'parameters': {
        'epochs': {
            'values': [10,15,20]
        },
        'learning_rate': {
            'values': [1e-2,1e-3]
        },
        'optimiser': {
            'values': ['gd','mgd','ngd', 'rmsprop','adam','nadam']
        },
        'activation': {
            'values': ['relu','tanh','sigmoid']
        },
        'hidden_layer': {
            'values': [[32,64],[64,64,64],[32,64,128],[128,128,128],[32,64,128,256]]
        },
        'batch_size':{
            'values':[64,128,256]
        },
        'weight_decay':{
            'values':[0.0,0.5,0.0005]
        },
        'loss_function':{
            'values':['cross_entropy']
        },
        'initilisation':{
            'values':['xavier']
        }
    }
}

In [10]:
sweep_id = wandb.sweep(sweep_config, entity="moni6264", project="test_100")

Create sweep with ID: 6f3f1wfl
Sweep URL: https://wandb.ai/moni6264/test_100/sweeps/6f3f1wfl


In [11]:
def train():
    #steps = 10
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'epochs': 5,
        'input_layer': 28*28,
        'output_layer': 10,
        'batch_size':64,
        'weight_decay':0.005,
        'learning_rate': 1e-3,
        'hidden_layer':[16,16,16],
        'learning_rate':1e-2,
        'optimiser':'mgd',
        'activation':'sigmoid',
        'initialisation':'xavier',
        'loss_function':'cross_entropy'
        
    }

    # Initialize a new wandb run
    wandb.init(project='test_100', entity='moni6264',config=config_defaults)
    
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    learning_rate = config.learning_rate
    epochs = config.epochs
    hidden_layer = config.hidden_layer
    activation = config.activation
    optimiser = config.optimiser
    input_layer = config.input_layer
    output_layer = config.output_layer
    batch_size = config.batch_size
    weight_decay = config.weight_decay
    loss_function = config.loss_function
    initialisation = config.initilisation
    #run_name = "init_{}_act_{}_lr_{}_opt_{}_L2_{}_bs_{}_ep_{}_hh_{}".format(initialisation,activation,learning_rate,optimiser, weight_decay,batch_size, epochs, hidden_layer)
    
    #wandb.run.name = run_name
    #wandb.run.save()
    #wandb.run.finish()
    # Model training here
    sweep_network    = ANN(input_layer, hidden_layer, output_layer,initialisation,activation,loss_function)
    acc1,acc2,train_loss,val_loss  = sweep_network.fit_predict(X_train,y_train,X_val,y_val,epochs,learning_rate,optimiser,batch_size,weight_decay)


In [12]:
#import warnings
#warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
#wandb.agent(sweep_id, train, count=100)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: uuscz40q with config:
wandb: 	activation: relu
wandb: 	batch_size: 256
wandb: 	epochs: 15
wandb: 	hidden_layer: [32, 64, 128]
wandb: 	initilisation: xavier
wandb: 	learning_rate: 0.01
wandb: 	loss_function: cross_entropy
wandb: 	optimiser: mgd
wandb: 	weight_decay: 0
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 0: training_accuracy = 63.20 %, training_loss= 1.06814  ,Validation accuracy = 63.15 ,Validation loss = 1.05832
Epoch 1: training_accuracy = 66.47 %, training_loss= 0.96141  ,Validation accuracy = 66.23 ,Validation loss = 0.95546
Epoch 2: training_accuracy = 67.86 %, training_loss= 0.91646  ,Validation accuracy = 67.85 ,Validation loss = 0.91236
Epoch 3: training_accuracy = 68.68 %, training_loss= 0.88952  ,Validation accuracy = 68.70 ,Validation loss = 0.88636
Epoch 4: training_accuracy = 69.35 %, training_loss= 0.87086  ,Validation accuracy = 69.42 ,Validation loss = 0.86821
Epoch 5: training_accuracy = 69.91 %, training_loss= 0.85683  ,Validation accuracy = 69.87 ,Validation loss = 0.85451
Epoch 6: training_accuracy = 70.33 %, training_loss= 0.84571  ,Validation accuracy = 70.47 ,Validation loss = 0.84362
Epoch 7: training_accuracy = 70.62 %, training_loss= 0.83656  ,Validation accuracy = 70.70 ,Validation loss = 0.83466
Epoch 8: training_accuracy = 70.88 %, training_loss= 0.8

steps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▄▅▅▆▆▇▇▇▇█████
train_loss,█▅▄▃▃▃▂▂▂▂▁▁▁▁▁
val_accuracy,▁▃▅▅▆▆▇▇▇▇█████
val_loss,█▅▄▃▃▃▂▂▂▂▁▁▁▁▁
steps,15
train_accuracy,71.863
train_loss,0.79838
val_accuracy,72.067
val_loss,0.79761


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 1bdfzszy with config:
wandb: 	activation: relu
wandb: 	batch_size: 64
wandb: 	epochs: 15
wandb: 	hidden_layer: [32, 64, 128, 256]
wandb: 	initilisation: xavier
wandb: 	learning_rate: 0.01
wandb: 	loss_function: cross_entropy
wandb: 	optimiser: rmsprop
wandb: 	weight_decay: 0.5
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 0: training_accuracy = 9.98 %, training_loss= 2.38644  ,Validation accuracy = 10.13 ,Validation loss = 2.39132
Epoch 1: training_accuracy = 9.98 %, training_loss= 2.37949  ,Validation accuracy = 10.13 ,Validation loss = 2.38436
Epoch 2: training_accuracy = 9.98 %, training_loss= 2.37195  ,Validation accuracy = 10.13 ,Validation loss = 2.37671
Epoch 3: training_accuracy = 9.98 %, training_loss= 2.36520  ,Validation accuracy = 10.13 ,Validation loss = 2.36979
Epoch 4: training_accuracy = 9.98 %, training_loss= 2.35942  ,Validation accuracy = 10.13 ,Validation loss = 2.36383
Epoch 5: training_accuracy = 9.98 %, training_loss= 2.35396  ,Validation accuracy = 10.13 ,Validation loss = 2.35815
Epoch 6: training_accuracy = 9.98 %, training_loss= 2.34883  ,Validation accuracy = 10.13 ,Validation loss = 2.35279
Epoch 7: training_accuracy = 9.98 %, training_loss= 2.34441  ,Validation accuracy = 10.13 ,Validation loss = 2.34814
Epoch 8: training_accuracy = 9.98 %, training_loss= 2.34035  ,Va

...

Epoch 0: training_accuracy = 20.50 %, training_loss= 2.27036  ,Validation accuracy = 20.62 ,Validation loss = 2.27011
Epoch 1: training_accuracy = 27.91 %, training_loss= 2.24535  ,Validation accuracy = 28.07 ,Validation loss = 2.24582
Epoch 2: training_accuracy = 33.62 %, training_loss= 2.22709  ,Validation accuracy = 33.15 ,Validation loss = 2.22798
Epoch 3: training_accuracy = 38.38 %, training_loss= 2.21198  ,Validation accuracy = 37.87 ,Validation loss = 2.21314
Epoch 4: training_accuracy = 41.87 %, training_loss= 2.19862  ,Validation accuracy = 40.88 ,Validation loss = 2.19998
Epoch 5: training_accuracy = 44.55 %, training_loss= 2.18635  ,Validation accuracy = 43.40 ,Validation loss = 2.18788
Epoch 6: training_accuracy = 46.54 %, training_loss= 2.17480  ,Validation accuracy = 45.10 ,Validation loss = 2.17646
Epoch 7: training_accuracy = 47.97 %, training_loss= 2.16372  ,Validation accuracy = 46.13 ,Validation loss = 2.16551
Epoch 8: training_accuracy = 49.05 %, training_loss= 2.1

steps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▂▄▅▅▆▆▇▇▇▇▇▇▇██████
train_loss,█▇▇▆▆▅▅▅▄▄▄▄▃▃▃▂▂▂▁▁
val_accuracy,▁▃▄▅▅▆▆▆▇▇▇▇▇▇██████
val_loss,█▇▇▆▆▅▅▅▄▄▄▄▃▃▃▂▂▂▁▁
steps,20
train_accuracy,55.22
train_loss,2.03858
val_accuracy,54.25
val_loss,2.04152


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: rrxy1chv with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 128
wandb: 	epochs: 20
wandb: 	hidden_layer: [64, 64, 64]
wandb: 	initilisation: xavier
wandb: 	learning_rate: 0.01
wandb: 	loss_function: cross_entropy
wandb: 	optimiser: gd
wandb: 	weight_decay: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 0: training_accuracy = 11.84 %, training_loss= 2.29545  ,Validation accuracy = 11.28 ,Validation loss = 2.29697
Epoch 1: training_accuracy = 12.62 %, training_loss= 2.29257  ,Validation accuracy = 11.80 ,Validation loss = 2.29401
Epoch 2: training_accuracy = 13.53 %, training_loss= 2.28973  ,Validation accuracy = 12.68 ,Validation loss = 2.29118
Epoch 3: training_accuracy = 14.42 %, training_loss= 2.28694  ,Validation accuracy = 13.63 ,Validation loss = 2.28841
Epoch 4: training_accuracy = 15.26 %, training_loss= 2.28421  ,Validation accuracy = 14.45 ,Validation loss = 2.28569
Epoch 5: training_accuracy = 16.09 %, training_loss= 2.28152  ,Validation accuracy = 15.10 ,Validation loss = 2.28301
Epoch 6: training_accuracy = 16.91 %, training_loss= 2.27888  ,Validation accuracy = 15.78 ,Validation loss = 2.28039
Epoch 7: training_accuracy = 17.67 %, training_loss= 2.27630  ,Validation accuracy = 16.55 ,Validation loss = 2.27781
Epoch 8: training_accuracy = 18.39 %, training_loss= 2.2

steps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▁▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
train_loss,██▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▁▁
val_accuracy,▁▁▂▂▃▃▃▄▄▄▅▅▅▆▆▇▇▇██
val_loss,██▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▁▁
steps,20
train_accuracy,25.365
train_loss,2.24875
val_accuracy,24.4
val_loss,2.25038
